In [77]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
# import warnings
# warnings.filterwarnings("ignore")
import sklearn
import pandas as pd
import numpy as np
import os

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [78]:
dr = 'exp/prepared/'
file_names = os.listdir(dr)

def lowerstr(f):
    for col in f:
        if '№ партии' in col:
            f['№ партии'] = f['№ партии'].apply(lambda x: str(x))            
            f['№ партии'] = f['№ партии'].str.lower()
        if '№ плавки' in col:
            f['№ плавки'] = f['№ плавки'].str.lower()
    return f


df_modes = pd.read_excel('merge_table_mean_chem.xlsx')
df_valid = pd.read_excel('valid_table_mean_chem.xlsx')
# table_stretching = pd.read_excel(dr+file_names[-2])
# table_stretching_valid = pd.read_excel(dr+file_names[-1])
# print(table_stretching.shape)
# print(table_stretching_valid.shape)
# 
# table_stretching_valid = lowerstr(table_stretching_valid)
table_stretching = pd.read_excel('table_stretching_13april.xlsx')
table_stretching = lowerstr(table_stretching)
df_valid['№ партии'] = df_valid['№ партии'].apply(lambda x: str(x).replace('.0', ''))

In [79]:
ls_all = [
    'Поставщик',
    'НД (гост, ту)',
    'ГОСТ, ТУ',
    'Дата форм. цехом',
    'марка стали',
    '№ плавки',
    'кол-во, шт.',
    '№ партии',
    '1 зона по ВТР закалка', 
    '1 зона уставка закалка',
    '1 зона фактическая температура закалка', 
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка', 
    '3 зона уставка закалка',
    '3 зона фактическая температура закалка',
    'Температура трубы на выгрузке из печи закалки',
    'Скорость прохождения трубы через спрейер, м/с',
    't˚ C трубы после спреера', 
    'Расход воды на закалочный спрейер, м3',
    'время выдержки в закалочной ванне, сек.',
    't˚ C воды в закалочной ванне', 
    't˚ C трубы после ванны',
    '1 зона ВТР и уставка отпуск', 
    '1 зона фактическая температура отпуск',
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '3 зона фактическая температура отпуск', 
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск', 
    '5 зона фактическая температура отпуск',
    'Температура трубы на выгрузке из печи отпуска',
    'шаг балок закалочная печь, сек', 
    'шаг балок отпускная печь, сек',
    'диаметр', 
    'толщина стенки', 
    'C', 
    'Mn', 
    'Si', 
    'P', 
    'S',
    'Cr', 
    'Ni', 
    'Cu', 
    'Al', 
    'V', 
    'Ti', 
    'Nb', 
    'Mo', 
    'N', 
    'Параметр',
    'Тип предела текучести 0.2',
    'Тип предела текучести 0.5',
    'Тип предела текучести 0.6',
    'C-coef',
    'ICD'
]

In [80]:
df_merge = pd.merge(df_modes, table_stretching, how ='left', on = ['№ плавки','№ партии'])
df_merge_valid = pd.merge(df_valid, table_stretching, how ='left', on = ['№ плавки','№ партии'])
chem = {#'Mn': 0, 'Si': 0, 
#         'P': 0, 
#         'Cr': 0, 'Ni': 0, 'Cu': 0, 'Al': 0, 'V': 0, 'Mo': 0, 
        'Ti': 0, 'Nb': 0, 'N': 0.004, 'S': 0}

df_merge = df_merge.fillna(value = chem)
df_merge_valid = df_merge_valid.fillna(value = chem)
df_merge.shape, df_merge_valid.shape

((21538, 50), (452, 39))

In [81]:
df_merge['Параметр закалка'] = 1/(1/(df_merge[['2 зона по ВТР закалка','3 зона по ВТР закалка']].mean(axis=1) 
                            + 273) - (2.303*1.986*np.log10(61*df_merge['шаг балок закалочная печь, сек']/3600)/110000))-273

df_merge_valid['Параметр закалка'] = 1/(1/(df_merge_valid[['2 зона по ВТР закалка','3 зона по ВТР закалка']].mean(axis=1) 
                            + 273) - (2.303*1.986*np.log10(61*df_merge_valid['шаг балок закалочная печь, сек']/3600)/110000))-273

In [82]:
df_merge['Параметр отпуск'] = (df_merge[['2 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273)*(20+np.log(94*df_merge['шаг балок отпускная печь, сек']/3600))*1e-3

df_merge_valid['Параметр отпуск'] = (df_merge_valid[['2 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273)*(20+np.log(94*df_merge_valid['шаг балок отпускная печь, сек']/3600))*1e-3

In [83]:
df_merge['Тип предела текучести (1186)'] = df_merge['Тип предела текучести (1186)'].apply(lambda x: str(x))
df_merge_valid['Тип предела текучести (1186)'] = df_merge_valid['Тип предела текучести (1186)'].apply(lambda x: str(x))

In [84]:
df_merge['Врем. сопротивление'] = df_merge['Врем. сопротивление'].apply(lambda x: str(x))
df_merge['Предел текучести'] = df_merge['Предел текучести'].apply(lambda x: str(x))
df_merge['t˚ C трубы после спреера'] = df_merge['t˚ C трубы после спреера'].apply(lambda x: str(x))
df_merge_valid['Врем. сопротивление'] = df_merge_valid['Врем. сопротивление'].apply(lambda x: str(x))
df_merge_valid['Предел текучести'] = df_merge_valid['Предел текучести'].apply(lambda x: str(x))
df_merge_valid['t˚ C трубы после спреера'] = df_merge_valid['t˚ C трубы после спреера'].apply(lambda x: str(x))

In [85]:
# ls_type_4 = [
#     'Тип предела текучести 0.2',
#     'Тип предела текучести 0.5',
#     'Тип предела текучести 0.6',
#     'del'
# ]

# ls_type_3 = [
#     'Тип предела текучести 0.2',
#     'Тип предела текучести 0.5',
#     'del'
# ]
# df_merge[ls_type_4] = pd.get_dummies(df_merge['Тип предела текучести (1186)'])
# df_merge_valid[ls_type_3] = pd.get_dummies(df_merge_valid['Тип предела текучести (1186)'])

In [86]:
df_merge = df_merge.reset_index(drop=True)
df_merge_valid = df_merge_valid.reset_index(drop=True)

In [87]:
df_merge = df_merge[df_merge['Cr'] < 10]
df_merge_valid = df_merge_valid[df_merge_valid['Cr'] < 10]

In [88]:
df_merge['C-coef'] = ((df_merge['C'] + df_merge['Mn'])/6.0 + (df_merge['Cr'] + df_merge['Mo'] + df_merge['V'])/5.0 +
                        (df_merge['Ni'] + df_merge['Cu'])/15.0)
df_merge_valid['C-coef'] = ((df_merge_valid['C'] + df_merge_valid['Mn'])/6.0 + (df_merge_valid['Cr'] + df_merge_valid['Mo'] + df_merge_valid['V'])/5.0 +
                        (df_merge_valid['Ni'] + df_merge_valid['Cu'])/15.0)

In [89]:
def ideal_critical_d(df):
    for x in df['C']:
        if x <= 0.39:
            coef1 = 0.54*x
        else:
            coef1 = 0.171+0.001*x+0.265*x*x
    for x in df['Mn']:
        if x <= 1.2:
            coef2 = 3.3333*x+1
        else:
            coef2 = 5.1*x-1.12
    result = coef1*coef2*(1+0.7*df['Si'])*(1+0.363*df['Ni'])*(1+2.16*df['Cr'])*(
            1+3.0*df['Mo'])*(1+0.365*df['Cu'])*(1+1.73*df['V'])
    df['ICD'] = result
    return df['ICD']

In [90]:
df_merge['ICD'] = ideal_critical_d(df_merge)
df_merge_valid['ICD'] = ideal_critical_d(df_merge_valid)

In [91]:
df_merge.shape, df_merge_valid.shape

((21518, 54), (452, 43))

In [92]:
# df_merge = df_merge[df_merge['Тип предела текучести 0.2'] == 1]

# df_merge_valid = df_merge_valid[df_merge_valid['Тип предела текучести 0.2'] == 1]

In [93]:
df_merge.shape, df_merge_valid.shape

((21518, 54), (452, 43))

In [94]:
def th2f(x):
    if x=='73.02*5.51':
        return 5.51
    if (x==' ') | (x==''):
        return None
    return float(x)
# tmp_modes[tmp_modes[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values('№ партии')
df_merge['толщина стенки'] = df_merge['толщина стенки'].apply(th2f)
df_merge = df_merge.groupby(['№ плавки', '№ партии']).first()
df_merge = df_merge.reset_index()
df_merge.shape

(21518, 54)

In [115]:
df_merge.to_excel('prepared_hard.xlsx')
df_merge_valid.to_excel('prepared_hard_valid.xlsx')

In [96]:
df_merge.shape

(21518, 54)

In [113]:
df_merge = to_kgs(df_merge)
df_merge_valid = to_kgs(df_merge_valid)

In [114]:
df_merge_valid['Врем. сопротивление'].describe()

count     452
unique     90
top       nan
freq      186
Name: Врем. сопротивление, dtype: object

In [106]:
def to_kgs(df_merge):
    df_merge['Предел текучести'] = df_merge['Предел текучести'].astype(float).apply(lambda x: x/9.8 if x>200 else x)
    df_merge['Предел текучести'] = df_merge['Предел текучести'].astype(float).apply(lambda x: x/9.8 if x>200 else x)
    return df_merge